# Lesson 16. SimPy &mdash; levels

### SA421 Fall 2015

* First, some setup code for the inverse transform method and SimPy.

In [ ]:
##### Setup for inverse transform method #####
# Import infinity from NumPy
from numpy import inf

# Import random number generator from NumPy
from numpy.random import rand

# Import bisect_right from bisect
from bisect import bisect_left

In [ ]:
##### Setup for SimPy #####
# Import seed initializer from NumPy
from numpy.random import seed

# Import all simulation functions from SimPy
from SimPy.Simulation import *

## Using SimPy to track inventory

**Problem.** The Simplex Bakery is trying to determine how many dozens of [cronuts](http://en.wikipedia.org/wiki/Cronut) to bake each day. 
The probability distribution of the number of cronut customers is as follows:

| Number of customers per day | 5    | 10   | 15   | 20   |
| --------------------------- | ---- | ---- | ---- | ---- |
| Probability                 | 0.15 | 0.40 | 0.35 | 0.10 |

Customers order 1, 2, 3 or 4 cronuts according to the following probability distribution:

| Number of dozen ordered per customer | 1   | 2   | 3   | 4   |
| ------------------------------------ | --- | --- | --- | --- |
| Probability                          | 0.3 | 0.4 | 0.2 | 0.1 |

Cronuts sell for \$3.50 per dozen. 
They cost \$2.00 per dozen to make. 
All cronuts not sold at the end of the day are sold at half price to a local stale baked good distributor.

Due to mixing size constraints, cronuts must be made in batches of 10 dozen.
Currently, the bakery bakes 20 dozen.

* Based on a 1-day simulation, what is the bakery's profit? How many customers are left unsatisfied (didn't get the number of cronuts wanted)?


* Based on a 1-day simulation, how many dozen should the bakery bake to maximize its profit?

* Let's start by creating functions that sample from the given probability distributions:

    - `D.customersPerDay` samples from the probability distribution on the number of customers per day.

    - `D.cronutsPerCustomer` that samples from the probability distribution on the number of cronuts each customer orders.

In [ ]:
##### Distributions #####
class D:
    # Number of customers per day
    def customersPerDay():
        # Sorted list of possible values and -inf
        a = [-inf, 5, 10, 15, 20]

        # cdf at each possible value
        cdf = [0, 0.15, 0.55, 0.90, 1]

        # Generate variate
        u = rand()
        i = bisect_left(cdf, u)
        variate = a[i]

        # Return generated variate
        return variate
    
    # Number of cronuts per customer
    def cronutsPerCustomer():
        # Sorted list of possible values and -inf
        a = [-inf, 1, 2, 3, 4]

        # cdf at each possible value
        cdf = [0, 0.30, 0.70, 0.90, 1]

        # Generate variate
        u = rand()
        i = bisect_left(cdf, u)
        variate = a[i]

        # Return generated variate
        return variate

* Let's also define input parameters for our simulation.

In [ ]:
##### Parameters #####
class P:
    # Revenue per dozen cronuts
    revenue = 3.5
    
    # Cost per dozen cronuts
    cost = 2.0
    
    # Number of dozen cronuts baked
    nCronuts = 20

## Levels

* We need a way to model the number of cronuts available at any given time.


* A **level** in SimPy represents inventory of a homogeneous, undifferentiated "material".
    - Levels can represent physical, discrete items, like cronuts.
    - Levels can also represent items of a continuous nature, like gasoline.
    - In addition, levels can represent more abstract items, such as packets in a computer network.


* Like with resources, we will put all levels in a common class called `L` for consistent naming purposes. 


* Also like with resources, we will create the SimPy level object later in the `model()` function that runs the simulation.

In [ ]:
##### Create dummy variable for cronut level #####
class L:


* Next, like with our other SimPy simulations, we need to define processes for our simulation model.


* Let's start by defining a `Customer` process that simulates the behavior of a customer.


* To get `q` units of material from a level, say `L.cronuts`, we use:
```
yield get, self, L.cronuts, q
```

* To add `q` units of material to a level, again, say `L.cronuts`, we use:
```
yield put, self, L.cronuts, q
```

* `L.cronuts.amount` gives the current amount held in the level `L.cronuts`.

In [ ]:
##### Create Customer process #####
class Customer(Process):
    def behavior(self):
        # How many cronuts are available?

        
        # Customer arrives

        
        # Customer determines how many cronuts she wants

        
        # Are there enough cronuts for this customer?


* Let's also define an `Entrance` process that models the arrival of customers to the bakery.

In [ ]:
# Entrance
class Entrance(Process):
    def behavior(self):
        # Determine how many customers will arrive at the bakery today
        nCustomersTotal = D.customersPerDay()
        
        # Generate customers
        nCustomersArrived = 0
        while nCustomersArrived < nCustomersTotal:
            # Create customer using the template defined in the Customer class
            c = Customer(name = "Customer {0}".format(nCustomersArrived))
            
            # Activate the customer's behavior
            activate(c, c.behavior())
            
            # Count this new customer
            nCustomersArrived += 1
            
            # Dummy yield hold statement
            yield hold, self, 0

* Note that processes in SimPy need at least 1 `yield` statement of some kind; hence the dummy `yield hold` statement above.

* Finally, let's create the `model()` function we customarily use to initialize SimPy, create any resources, levels, and monitors, and run the simulation.


* Let's have the `model()` function take as input the seed we want to use to initialize the random number generator.


* In addition, <span style="color:#d00000">let's have the `model()` function output, or return, the resulting profit of the day.</span>

In [ ]:
def model(inputSeed):
    # Initialize SimPy
    initialize()
    
    # Initialize a seed for the random number generator
    seed(inputSeed)

    # Create level for cronuts

    
    # Activate the entrance
    e = Entrance()
    activate(e, e.behavior())
    
    # Run the simulation
    simulate(until = inf)

    # Compute profit

    
    # Return performance measure
    return profit

* A level is created by setting the level's variable name to a call of the `Level()` function.
    - `initialBuffered` is the initial quantity of material in the level when it is created.
    

* Note that we don't have a predetermined time limit. On the other hand, by design, we have a finite number of customers based on `D.customersPerDay()`. 


* By using `until = inf` when calling `simulate()`, SimPy will instead terminate the simulation when there are no more customers to process.


* Let's run the model and see what happens.

In [ ]:
# Run the model once
model(123)

* Note that you can integrate levels with resources in a SimPy model to model complex situations where queueing and inventory interact.

## With a neighbor...

* Implement a monitor called `M.satisfied` that records for each customer:
    - 1 if the customer receives his/her entire order
    - 0 otherwise.
    
    
* Use this monitor to compute the fraction of customers that are left unsatisfied.

* First, establish a class `M` to hold our monitors and define a dummy variable for `M.satisfied`.

In [ ]:
##### Declare dummy variables for new monitors #####
class M:
  

* Redefine `model()` to activate the monitor.

In [ ]:
##### Activate new monitors #####


* Modify the customer behavior so that the monitor observes the appropriate values.

In [ ]:
##### Change Customer process to record satisfaction (0 or 1) using newly activated monitors #####


* Run the model and compute the desired performance measure.

In [ ]:
# Run the model once
model(123)

In [ ]:
##### Compute fraction of customers that are unsatisfied #####
print("Fraction of customers unsatisfied = {0}".format(1 - M.satisfied.mean()))

## Optimizing cronut production

* Finally, let's determine the number of cronuts the Simplex Bakery needs to produce to maximize its profits.


* Before we begin, let's modify the `Customer` process so that it does not print all the details.

In [ ]:
# Customer
class Customer(Process):
    def behavior(self):
        # Customer arrives
        # Customer determines how many cronuts she wants
        cronutsWanted = D.cronutsPerCustomer()
   
        # Are there enough cronuts for this customer?
        if cronutsWanted > L.cronuts.amount:
            yield get, self, L.cronuts, L.cronuts.amount
        else:
            yield get, self, L.cronuts, cronutsWanted

* Now, let's run the model for differing numbers of cronuts baked.

* Based on one replication, what is the optimal number of cronuts?

*Write your notes here. Double-click to edit.*